In [1]:
from src.data.tushare.model.cne5 import TuShareCNE5_Calculator

task_cne5 = TuShareCNE5_Calculator()
task_cne5.calc_specific_risk(20241104)

,secid,spec_risk
0,1,0.012616
1,2,0.016491
2,4,0.076816
3,6,0.058301
4,7,0.058771
...,...,...
5298,873339,0.084023
5299,873527,0.075710
5300,873576,NaN
5301,873593,NaN


In [78]:
from src_factor_definition.level0.volatility_skew import price_weiskew12m
price_weiskew12m().calc_factor(20241104)


secid
1         1.422159
2         1.300609
4         0.426441
5              NaN
6         0.155944
            ...   
920019         NaN
920066         NaN
920088         NaN
920099    0.961194
920118    1.120213
Length: 5415, dtype: float64

In [1]:
from src.data.tushare.download.daily import DailyMoneyFlow

dmf = DailyMoneyFlow()
dmf.update_dates()



array([20100102, 20100103, 20100104, ..., 20241103, 20241104, 20241105])

In [1]:
from src.data import TSData
TSData.TRADE.get_mv(20241029 , 20241104)


circ_mv
secid  date                  
1      20241029  2.258814e+11
       20241030  2.239408e+11
       20241031  2.196716e+11
       20241101  2.208359e+11
       20241104  2.218062e+11
...                       ...
920118 20241029  3.617049e+08
       20241030  3.830871e+08
       20241031  4.164711e+08
       20241101  3.764656e+08
       20241104  3.448750e+08

[26733 rows x 1 columns]

In [8]:
import tushare as ts
pro = ts.pro_api('2026c96ef5fa7fc3241c96baafd638c585284c7fefaa00b93ef0a62c')
df = ts.pro_bar(ts_code='000001.SZ', api = pro , adj='qfq', start_date='20180101', end_date='20181011')

c:\Users\jinmeng\AppData\Local\Programs\Python\Python311\Lib\site-packages\tushare\pro\data_pro.py:131: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['adj_factor'] = data['adj_factor'].fillna(method='bfill')


In [9]:
df

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20181011,10.05,10.16,9.70,9.86,10.45,-0.59,-5.65,1995143.83,1994186.611
1,000001.SZ,20181010,10.54,10.66,10.38,10.45,10.56,-0.11,-1.04,995200.08,1045666.180
2,000001.SZ,20181009,10.46,10.70,10.39,10.56,10.45,0.11,1.05,1064084.26,1117946.550
3,000001.SZ,20181008,10.70,10.79,10.45,10.45,11.05,-0.60,-5.43,1686358.52,1793455.283
4,000001.SZ,20180928,10.78,11.27,10.78,11.05,10.74,0.31,2.89,2110242.67,2331358.288
...,...,...,...,...,...,...,...,...,...,...,...
182,000001.SZ,20180108,13.04,13.08,12.66,12.75,13.09,-0.34,-2.60,2158620.81,2806099.169
183,000001.SZ,20180105,13.00,13.14,12.94,13.09,13.04,0.05,0.38,1210312.72,1603289.517
184,000001.SZ,20180104,13.11,13.16,12.92,13.04,13.12,-0.08,-0.61,1854509.48,2454543.516
185,000001.SZ,20180103,13.51,13.64,12.99,13.12,13.48,-0.36,-2.67,2962498.38,4006220.766


In [13]:
import tushare as ts

pro = ts.pro_api('2026c96ef5fa7fc3241c96baafd638c585284c7fefaa00b93ef0a62c')
df = ts.pro_bar(ts_code='600000.SH', api = pro , freq='1min', 
                start_date='2024-11-05 00:00:00', 
                end_date='2024-11-05 17:00:00')

df

In [14]:
df

,ts_code,trade_time,close,open,high,low,vol,amount,trade_date,pre_close,change,pct_chg
0,600000.SH,2024-11-05 15:00:00,10.06,10.06,10.06,10.06,1196900.0,12040814.0,20241105,10.06,0.00,0.0
1,600000.SH,2024-11-05 14:59:00,10.06,10.06,10.06,10.06,0.0,0.0,20241105,10.06,0.00,0.0
2,600000.SH,2024-11-05 14:58:00,10.06,10.06,10.06,10.06,2000.0,20120.0,20241105,10.05,0.01,0.1
3,600000.SH,2024-11-05 14:57:00,10.05,10.05,10.06,10.04,185072.0,1860740.0,20241105,10.05,0.00,0.0
4,600000.SH,2024-11-05 14:56:00,10.05,10.05,10.06,10.04,200823.0,2018952.0,20241105,10.06,-0.01,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...
236,600000.SH,2024-11-05 09:34:00,9.97,9.98,9.98,9.95,193258.0,1925624.0,20241105,9.97,0.00,0.0
237,600000.SH,2024-11-05 09:33:00,9.97,9.99,9.99,9.97,204800.0,2044846.0,20241105,9.99,-0.02,-0.2
238,600000.SH,2024-11-05 09:32:00,9.99,10.00,10.01,9.99,374486.0,3744104.0,20241105,9.99,0.00,0.0
239,600000.SH,2024-11-05 09:31:00,9.99,9.95,10.00,9.93,473144.0,4715288.0,20241105,9.97,0.02,0.2


In [1]:
from src.data.tushare.model.cne5 import TuShareCNE5_Calculator
cne5 = TuShareCNE5_Calculator()
cne5.Update('exposure')
cne5.Update('risk')


In [2]:
from src.data.tushare.download.daily import DailyMoneyFlow
DailyMoneyFlow().get_data(20241104)

,secid,buy_sm_vol,buy_sm_amount,sell_sm_vol,sell_sm_amount,buy_md_vol,buy_md_amount,sell_md_vol,sell_md_amount,buy_lg_vol,buy_lg_amount,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount
0,1,206641,23472.72,214673,24397.62,294219,33423.61,300228,34133.57,349891,39772.04,351405,39933.79,280765,31919.03,265211,30122.43,-66602,-7439.96
1,2,557125,49903.84,383794,34410.17,636053,56992.08,573613,51429.29,496666,44509.93,574776,51486.75,247204,22159.22,404864,36238.86,-182446,-16153.23
2,4,75115,13725.40,64391,11822.04,45394,8307.64,52668,9630.07,26685,4909.84,24742,4527.00,3369,622.08,8762,1585.84,2299,428.43
3,6,286767,24512.43,197046,16857.81,292197,24981.22,317110,27118.47,230250,19685.66,277444,23679.77,96853,8279.79,114466,9803.05,-93641,-7936.06
4,7,47080,3313.73,54514,3838.73,52907,3730.79,44947,3173.40,29532,2082.85,30665,2159.18,2176,154.71,1569,110.77,27082,1912.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5088,688799,3813,1498.85,3640,1431.49,5319,2090.90,5359,2107.46,1958,770.76,2090,821.56,0,0.00,0,0.00,-763,-297.29
5089,688800,24307,8921.19,25042,9211.46,18769,6904.80,19942,7339.27,11353,4183.04,7961,2920.42,2903,1077.74,4387,1615.61,2069,773.56
5090,688819,29010,8197.77,28051,7928.01,20181,5708.49,19625,5551.70,8531,2411.08,10370,2932.08,992,282.18,668,187.73,-4120,-1163.00
5091,688981,72765,62502.24,84901,72982.84,181190,155670.85,183621,157781.59,215116,184889.83,220352,189364.82,107512,92388.61,87708,75322.27,12858,11539.99


In [ ]:
import numpy as np
import pandas as pd

from src.factor.classes import StockFactorCalculator
from src.data import TSData

def amount_coefficient_variance(date , n_months : int , lag_months : int = 0 , min_finite_ratio = 0.25):
    start_date , end_date = TSData.CALENDAR.td_start_end(date , n_months , 'm' , lag_months)
    amts = TSData.TRADE.mask_min_finite(TSData.TRADE.get_volumes(start_date , end_date , volume_type = 'amount'))
    amt_cv = amts.std() / amts.mean()
    return amt_cv

class amt_cv1m(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'liquidity'
    description = '资金流--收益率划'
    
    def calc_factor(self, date: int):
        return amount_coefficient_variance(date , 1)

from cProfile import Profile
with Profile() as pr:
    v = amt_cv1m().calc_factor(20241104)

In [7]:
import numpy as np
import pandas as pd

from src.factor.classes import StockFactorCalculator
from src.data import TSData

def exret_std(date , n_months : int , lag_months : int = 0):
    start_date , end_date = TSData.CALENDAR.td_start_end(date , n_months , 'm' , lag_months)
    exrets = TSData.MODEL.get_exret(start_date , end_date , pivot=True)
    return exrets.std() * np.sqrt(252)

class exret_std1m(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'volatility'
    description = '1个月收益率标准差'
    
    def calc_factor(self, date: int):
        return exret_std(date , 1)

from cProfile import Profile
with Profile() as pr:
    v = exret_std1m().calc_factor(20241104) 
# pr.print_stats(1)
v

secid
1         0.188706
2         0.263073
4         0.773525
6         1.062064
7         0.644802
            ...   
873339    0.886305
873527    0.790199
873576    0.946141
873593    1.349053
873665    1.196075
Length: 5150, dtype: float64

In [26]:
from src.api.factor import factor_hierarchy
hier = factor_hierarchy()
hier.factor_df()




,level,file_name,factor_name,init_date,category0,category1,description
0,level0,correlation_beta,beta_12m,20070101,behavior,correlation,12个月贝塔
1,level0,correlation_beta,beta_1m,20070101,behavior,correlation,1个月贝塔
2,level0,correlation_beta,beta_2m,20070101,behavior,correlation,2个月贝塔
3,level0,correlation_beta,beta_3m,20070101,behavior,correlation,3个月贝塔
4,level0,correlation_beta,beta_6m,20070101,behavior,correlation,6个月贝塔
5,level0,liquidity_amtcv,amt_cv12m,20070101,behavior,liquidity,12个月成交额变异系数
6,level0,liquidity_amtcv,amt_cv1m,20070101,behavior,liquidity,1个月成交额变异系数
7,level0,liquidity_amtcv,amt_cv2m,20070101,behavior,liquidity,2个月成交额变异系数
8,level0,liquidity_amtcv,amt_cv3m,20070101,behavior,liquidity,3个月成交额变异系数
9,level0,liquidity_amtcv,amt_cv6m,20070101,behavior,liquidity,6个月成交额变异系数


In [3]:
date = 20241031
factor_values = hier.test_calc_all_factors(date)

beta_12m calculated
beta_1m calculated
beta_2m calculated
beta_3m calculated
beta_6m calculated
amt_cv12m calculated
amt_cv1m calculated
amt_cv2m calculated
amt_cv3m calculated
amt_cv6m calculated
bsact_neg_1m calculated
bsact_pos_1m calculated
turn_12m calculated
turn_1m calculated
turn_2m calculated
turn_3m calculated
turn_6m calculated
mom_12m calculated
mom_12m_1m calculated
mom_1m calculated
mom_1m_intraday calculated
mom_1m_overnight calculated
mom_2m calculated
mom_3m calculated
mom_6m calculated
mom_mdr12m calculated
mom_mdr1m calculated
mom_mdr2m calculated
mom_mdr3m calculated
mom_mdr6m calculated
mom_phigh12m calculated
mom_phigh1m calculated
mom_phigh2m calculated
mom_phigh3m calculated
mom_phigh6m calculated
exret_std12m calculated
exret_std1m calculated
exret_std2m calculated
exret_std3m calculated
exret_std6m calculated
ret_std12m calculated
ret_std1m calculated
ret_std2m calculated
ret_std3m calculated
ret_std6m calculated


In [8]:
cls = hier.get_factor('beta_12m')
cls().calc_factor(20241029)

secid
1         0.793899
2         1.321322
4         1.306744
5         0.365365
6         1.030893
            ...   
873339    1.385271
873527    1.393108
873576    1.681234
873593    2.240738
873665    1.610474
Length: 5289, dtype: float64

In [2]:
hier.factor_df(level='level0')

,level,factor_name,init_date,category0,category1,description
0,level0,ret20_raw,20070101,behavior,momentum,raw return of 20 days


In [3]:
hier.update_jobs(20241001)
hier.jobs()

,date,level,factor
0,20241008,level0,"ret20_raw(from 20070101 , behavior , momentum)"
1,20241009,level0,"ret20_raw(from 20070101 , behavior , momentum)"
2,20241010,level0,"ret20_raw(from 20070101 , behavior , momentum)"
3,20241011,level0,"ret20_raw(from 20070101 , behavior , momentum)"
4,20241014,level0,"ret20_raw(from 20070101 , behavior , momentum)"
5,20241015,level0,"ret20_raw(from 20070101 , behavior , momentum)"
6,20241016,level0,"ret20_raw(from 20070101 , behavior , momentum)"
7,20241017,level0,"ret20_raw(from 20070101 , behavior , momentum)"
8,20241018,level0,"ret20_raw(from 20070101 , behavior , momentum)"
9,20241021,level0,"ret20_raw(from 20070101 , behavior , momentum)"
